In [2]:
import math
from pathlib import Path

import arcpy
from ba_tools import data
import pandas as pd
import numpy as np

from h3 import h3
from itertools import chain
import json
import os
from pathlib import Path
import sys
from copy import deepcopy

from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import pandas as pd

project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

data_raw = data_dir/'raw'
data_int = data_dir/'interim'
data_out = data_dir/'processed'

gdb_raw = data_raw/'raw.gdb'
gdb_int = data_int/'interim.gdb'
gdb_out = data_out/'processed.gdb'

# import the project package from the project package path
sys.path.append(str(project_parent/'src'))
import h3_arcgis

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

In [11]:
# pth_bp = Path(r'D:\arcgis\ba_data\Data\Demographic Data\block_data.gdb\USA_ESRI_2019_blocks')
pth_bp = gdb_int/'block_points_us_contermonious'
pth_bp

df_bp = GeoAccessor.from_featureclass(str(pth_bp))
df_bp.head()

df_bp = df_bp[['FIPS', 'SHAPE', 'HH_C', 'POP_C']].copy()
df_bp.columns = ['FIPS', 'SHAPE', 'HouseholdCount', 'Population']
df_bp.head()

df_bp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6873144 entries, 0 to 6873143
Data columns (total 4 columns):
FIPS              object
SHAPE             geometry
HouseholdCount    float64
Population        float64
dtypes: float64(2), geometry(1), object(1)
memory usage: 209.8+ MB


In [12]:
threshold_count = 1000
count_column = 'Population'

df_ids = h3_arcgis.get_nonoverlapping_h3_hexbins_for_points(
        spatially_enabled_dataframe=df_bp,
        h3_minimum=5,
        h3_maximum=9,
        minimum_threshold_value=threshold_count,
        weighting_column=count_column
    )

In [13]:
df_ids.spatial.to_featureclass(str(gdb_int/f'h3_{count_column.lower()}_{threshold_count}_nat'))
df_ids.to_csv(data_int/f'h3_{count_column.lower()}_{threshold_count}_05.csv')